<a href="https://colab.research.google.com/github/chirag2saraiya/Image/blob/master/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!rm -rf /content/images

In [0]:
!mkdir images


In [0]:
!mkdir Cars

In [0]:
!pip install numpy scipy pillow

In [0]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.preprocessing import image
from keras import applications
from PIL import Image
import os
import pickle
import sys
#import tensorlayer as tl
#from scipy.misc import imsave
from keras.preprocessing.image import save_img

import matplotlib.pyplot as plt

import sys

import numpy as np

# Input shape
img_rows = 96
img_cols = 96
channels = 3
img_shape = (img_rows, img_cols, channels)
latent_dim = 96
data_dir = "/content/cars_train"
optimizer = Adam(0.0002, 0.5)
#g_loss = np.empty(shape=1)
#d_loss_real = np.empty(shape=1)
#d_loss_fake = np.empty(shape=1)

def build_generator():
  model = Sequential()
  model.add(Dense(100, activation="relu", input_dim=latent_dim))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Activation("relu"))
  model.add(Dense(512*6*6))
  model.add(BatchNormalization())
  model.add(Activation("relu"))
  model.add(Reshape((6, 6, 512),input_shape=(512*6*6,)))
  model.add(UpSampling2D())
  model.add(Conv2D(256, kernel_size=3, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Activation("relu"))
  model.add(UpSampling2D())
  model.add(Conv2D(128, kernel_size=3, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Activation("relu"))
  model.add(UpSampling2D())
  model.add(Conv2D(64, kernel_size=3, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Activation("relu"))
  model.add(UpSampling2D())
  model.add(Conv2D(32, kernel_size=3, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  #model.add(Activation("relu"))
  model.add(Conv2D(channels, kernel_size=3, padding="same"))
  model.add(Activation("tanh"))

  model.summary()

  noise = Input(shape=(latent_dim,))
  img = model(noise)

  return Model(noise, img)

def build_discriminator():

  model = Sequential()

  model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
  model.add(ZeroPadding2D(padding=((0,1),(0,1))))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  model.add(Conv2D(256, kernel_size=3, strides=2, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid'))

  model.summary()

  img = Input(shape=img_shape)
  validity = model(img)

  return Model(img, validity)
        
# Build and compile the discriminator
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
optimizer=optimizer,
metrics=['accuracy'])

# Build the generator
generator = build_generator()

# The generator takes noise as input and generates imgs
z = Input(shape=(latent_dim,))
img = generator(z)

# For the combined model we will only train the generator
discriminator.trainable = False

# The discriminator takes generated images as input and determines validity
valid = discriminator(img)

# The combined model  (stacked generator and discriminator)
# Trains the generator to fool the discriminator
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

data_generator = image.ImageDataGenerator(preprocessing_function=applications.xception.preprocess_input,
    data_format='channels_last',
    rotation_range=180.0,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    channel_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True
    )
  
real_image_generator = data_generator.flow_from_directory(
    directory=data_dir,
    target_size=(96, 96),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)


def save_imgs( epoch):
  r, c = 5, 5
  fixed_noise = np.random.normal(size=(32, latent_dim))
  gen_imgs = generator.predict(fixed_noise)
  # Rescale images 0 - 1
  gen_imgs = 0.5 * gen_imgs + 0.5
  filepath2="{}{}.{}".format(F"/content/images/",epoch,".png")
  print(type(gen_imgs))
  print(gen_imgs.shape)
  print(gen_imgs[0].shape)
  print(type(gen_imgs[0]))
  save_img(filepath2, gen_imgs[0])
  #Image.fromarray(gen_imgs.save("/content/images/car_%d.png" % epoch)
  #filepath2="/content/images/car_%d.png".format(epoch)
  #save_img(filepath2, gen_imgs)
  #Image.fromarray(gen_imgs[0], mode='RGB').save(os.path.join('/content/images/', 'car_{}.png').format(epoch))
  #fig, axs = plt.subplots(r, c)
  #cnt = 0
  #for i in range(r):
  #  for j in range(c):
  #    axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
  #    axs[i,j].axis('off')
  #    cnt += 1
  #fig.savefig("/content/images/car_%d.png" % epoch)
  #plt.close() 
  
        
def get_image_batch(batch_size):    
  
  #img_batch = real_image_generator.next()
  print("==========================")
  
  
  #print(type(img_batch))
  # keras generators may generate an incomplete batch for the last batch in an epoch of data
  

  imgs = real_image_generator.next()[0]
  #print(imgs.shape[0])
  if imgs.shape[0] != batch_size:
    imgs = real_image_generator.next()[0]
  assert imgs.shape == (batch_size, img_rows, img_cols, channels), imgs.shape
  #print(type(imgs))
  #print(imgs.shape)
  return imgs   

def train(epochs, batch_size=128, save_interval=50):
    # Load the dataset
    #(X_train, _), (_, _) = mnist.load_data()
    #Rescale -1 to 1
    #X_train = X_train / 127.5 - 1.
    #X_train = np.expand_dims(X_train, axis=3)

    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random half of images
        #idx = np.random.randint(0, X_train.shape[0], batch_size)
        #imgs = X_train[idx]
        #print(imgs.shape)


        # Sample noise and generate a batch of new images
        #noise = np.random.normal(0, 1, (batch_size, latent_dim))
        noise = np.random.normal(size=(batch_size, latent_dim))
        gen_imgs = generator.predict(noise)

        x = get_image_batch(batch_size)

        
        #x=np.asarray(x)
        #print(type(imgs))
        #print(type(x))
        
        # Train the discriminator (real classified as ones and generated as zeros)
        d_loss_real = discriminator.train_on_batch(x, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---------------------
        #  Train Generator
        # ---------------------

        # Train the generator (wants discriminator to mistake images as real)
        g_loss = combined.train_on_batch(noise, valid)
        g_loss = combined.train_on_batch(noise, valid)
        g_loss = combined.train_on_batch(noise, valid)

        # Plot the progress
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

        # If at save interval => save generated image samples
        if epoch % save_interval == 0:
            save_imgs(epoch)

def main():
    train(epochs=4000, batch_size=32, save_interval=50)
if __name__ == '__main__':
    main()

In [0]:
!mv /*.JPG /content/Cars/

In [0]:
!mkdir /content/content/smallCar/cars
!mv /content/content/smallCar/* /content/content/smallCar/cars

mv: cannot move '/content/content/smallCar/cars' to a subdirectory of itself, '/content/content/smallCar/cars/cars'


In [0]:
!mkdir /content/cars_train/cars

In [0]:
!mv /content/cars_train/*.jpg /content/cars_train/cars/

In [0]:
!mkdir /content/content/smallCar/cars
!mv /content/sample_data/* /content/content/smallCar/cars

mkdir: cannot create directory ‘/content/content/smallCar/cars’: No such file or directory
mv: target '/content/content/smallCar/cars' is not a directory


In [0]:
import Image
import os
import sys

directory = sys.argv[1]

for file_name in os.listdir(directory):
  print("Processing %s" % file_name)
  image = Image.open(os.path.join(directory, file_name))

  x,y = image.size
  new_dimensions = (x/2, y/2)
  output = image.resize(new_dimensions, Image.ANTIALIAS)

  output_file_name = os.path.join(directory, "small_" + file_name)
  output.save(output_file_name, "JPEG", quality = 95)

print("All done")

In [0]:
!cd /content
!mkdir smallCar

In [0]:
from PIL import Image
import os
import sys

directory = '/content/Cars'

for file_name in os.listdir(directory):
  print("Processing %s" % file_name)
  image = Image.open(os.path.join(directory, file_name))

  #x,y = image.size
  new_dimensions = (100, 100)
  output = image.resize(new_dimensions, Image.ANTIALIAS)

  output_file_name = os.path.join('/content/smallCar', file_name)
  output.save(output_file_name, "JPEG", quality = 95)

print("All done")

In [0]:
!zip -r smallCar.zip /content/smallCar


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp /content/smallCar.zip /content/drive/'My Drive'/

In [0]:
!cp /content/drive/'My Drive'/smallCar.zip /content/


In [0]:
!cp /content/drive/'My Drive'/cars_train.tgz /content/

In [0]:
!tar -xzf /content/cars_train.tgz

In [0]:
!zip -r images.zip /content/images

In [0]:
!cp /content/images.zip /content/drive/'My Drive'/

In [0]:
!unzip /content/smallCar.zip

Archive:  /content/smallCar.zip
   creating: content/smallCar/
  inflating: content/smallCar/68.f1223716690ace01ef686c6298191d31.jpg  
  inflating: content/smallCar/31.main_cropped.jpg  
  inflating: content/smallCar/63.Top-SUVs-Cars.jpg  
  inflating: content/smallCar/12.1.jpg  
  inflating: content/smallCar/16.a3a808220059e603efc8e21d8dc2a441.jpg  
  inflating: content/smallCar/1.Brezza.jpg  
  inflating: content/smallCar/26.925065586s.jpg  
  inflating: content/smallCar/48.85237-3ed53e7dcf9668dc1ac62eb3d12a4717.jpg  
  inflating: content/smallCar/33.925732990s.jpg  
  inflating: content/smallCar/74.main_cropped.jpg  
  inflating: content/smallCar/73.AudiA3-171966505_6.jpg  
  inflating: content/smallCar/17.IMG_2011.jpg  
  inflating: content/smallCar/14.1411559612_2014-renault-duster-awd-launched-india-price-feature-details.jpg  
  inflating: content/smallCar/50.jeepwrangler-1000_6.jpg  
  inflating: content/smallCar/34.Maruti-Suzuki-Swift-2-web.jpg  
  inflating: content/smallCar/6

In [0]:
!ls -l /content/drive/'My Drive'/smallCar.zip

-rw------- 1 root root 4738764 Nov 14 19:42 '/content/drive/My Drive/smallCar.zip'
